# DECISION  TREE CLASSIFIER ---------------------------------------------------------------------

### Get Train, Valid, Test data

In [ ]:
import pandas as pd
from my_libs import lib_tools as pt

# run_type = 'dev'
run_type = 'prd'
gen_sample = True
find_best_params = False

X_train, y_train, X_valid, y_valid, X_test, y_test = pt.get_train_valid_test_data(run_type)
print("Train, valid and Test data loaded")

### Resample data with SMOTEN()

In [ ]:
if gen_sample:
    X_train_rs, y_train_rs = pt.get_data_resampled(X=X_train, y=y_train, verbose=1)
    # Save data generated
    X_train_rs.to_pickle(f'./pickles/X_train_smote_{run_type}_dt_{X_train.shape[0]}.pkl')
    y_train_rs.to_pickle(f'./pickles/y_train_smote_{run_type}_dt_{X_train.shape[0]}.pkl')
else:
    # Load data previously generated
    X_train_rs = pd.read_pickle(f'./pickles/X_train_smote_{run_type}_dt_{X_train.shape[0]}.pkl')
    y_train_rs = pd.read_pickle(f'./pickles/y_train_smote_{run_type}_dt_{X_train.shape[0]}.pkl')

pt.plot_data_augmentation(y_train, y_train_rs)

### Encode categorical data (target and one hot encoding)

In [ ]:
from my_libs.encoder_custom import  EncoderCustom

cols_target_encoded = ['dep']
cols_onehot_encoded = X_train.columns.drop(cols_target_encoded)

encoder = EncoderCustom(cols_target_encoded=cols_target_encoded, cols_onehot_encoded=cols_onehot_encoded)

X_train_rs_enc, y_train_rs_enc = encoder.transform(X=X_train_rs, y=y_train_rs, datatype='Train')
X_valid_enc, y_valid_enc = encoder.transform(X=X_valid, y=y_valid, datatype='Test')
X_test_enc, y_test_enc = encoder.transform(X=X_test , y=y_test , datatype='Test')

### Find best hyperparameters for model with Optuna

In [ ]:
import time
import optuna
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

if find_best_params:

    n_trials = 10
    start_time = time.time()

    def objective(trial):

        dt_criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
        dt_splitter = trial.suggest_categorical('splitter', ['best', 'random'])
        dt_max_depth = trial.suggest_int('max_depth', 2, 20)
        dt_min_samples_split = trial.suggest_int('min_samples_split', 2, 50)
        classifier_obj = DecisionTreeClassifier(criterion=dt_criterion, splitter=dt_splitter, max_depth=dt_max_depth, min_samples_split=dt_min_samples_split)

        score = cross_val_score(classifier_obj, X_train_rs_enc, y_train_rs_enc, cv=5, scoring="f1", verbose=1)
        accuracy = score.mean()

        return accuracy

    # Create a study object and optimize the objective function.
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    print(f"\n--- Decision Tree Classifier - Optimization with Optuna performed in %s seconds ---" % (time.time() - start_time))
    print(f"Best params : {study.best_params}")
    
    from optuna.visualization import plot_optimization_history
    fig = plot_optimization_history(study)
    fig.show()

### Fit best model

In [ ]:
if find_best_params:
    # model coming from optuna search
    params = study.best_params    
else:
    # train best model    
    criterion = 'gini'
    splitter = 'best'
    max_depth = 16
    min_samples_split = 11
    params = {'criterion': criterion, 'splitter': splitter, 'max_depth': max_depth, 'min_samples_split': 5}
    
model = DecisionTreeClassifier(**params)   
model.fit(X_train_rs_enc, y_train_rs_enc);

###  Plot Train and Test learning curves

In [ ]:
import numpy as np
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
%matplotlib inline

# Compute the learning curve
train_sizes, train_scores, valid_scores = learning_curve(model, X_train_rs_enc, y_train_rs_enc, cv=5, scoring='roc_auc', n_jobs=-1)

# Calculate the mean and standard deviation of the training and validation scores
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std  = np.std(train_scores, axis=1)
valid_scores_mean = np.mean(valid_scores, axis=1)
valid_scores_std  = np.std(valid_scores, axis=1)

# Plot the learning curve
plt.figure(figsize=(8, 6))
plt.title('Decision Tree Classifier Learning Curve')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy Score')
plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color='r')
plt.fill_between(train_sizes, valid_scores_mean - valid_scores_std, valid_scores_mean + valid_scores_std, alpha=0.1, color='g')
plt.plot(train_sizes, train_scores_mean, 'o-', color='r', label='Training Score')
plt.plot(train_sizes, valid_scores_mean, 'o-', color='g', label='Validation Score')
plt.legend(loc='best')
plt.show()


### Confusion Matrix


In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test_enc)    
display(pd.crosstab(y_test_enc, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite']))
print("\nClassification report -------------------------------\n")
print(classification_report(y_test_enc, y_pred))

### Plot ROC curves (from estimator & from predictions)

In [ ]:
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt
import numpy as np

fig, axs = plt.subplots(1,2, figsize=(20,10))

RocCurveDisplay.from_estimator(model, X_test_enc, y_test_enc, ax=axs[0])
axs[0].plot(np.arange(0, 1, 0.01), np.arange(0, 1, 0.01))
axs[0].set_title('Gradient Boosting - ROC Curve from estimator')
axs[0].grid(linestyle='--')

RocCurveDisplay.from_predictions(y_test_enc, model.predict(X_test_enc), ax=axs[1])
axs[1].plot(np.arange(0, 1, 0.01), np.arange(0, 1, 0.01))
axs[1].set_title('Gradient Boosting - ROC Curve from predictions');
axs[1].grid(linestyle='--')

### Save model to h5

In [ ]:
from joblib import dump
# Save the model to an h5 file using joblib
dump(model, f'h5_models/model_dt_{run_type}_{X_train.shape[0]}.h5')